In [1]:
import json
import base64
import qrcode
import requests
import datetime
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [2]:
secrets_resolver = SecretsResolverDemo()

In [3]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [4]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            print(private_key)
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            print(private_key)
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Alice get OOB from Mediator

In [41]:
# oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text

In [42]:
print(oob_url)

https://mediator.rootsid.cloud?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiYTJmYzI4NjMtZjhjNi00N2Y5LWEzZDMtODg3MWNlNzA0YmFiIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNvN2dyZUFRNEdaUnVwYXJRQzFjeXN4dWJjQ0ZqVlZaZHN2RFNvY0VrTDk3MS5WejZNa3RpRmg3aDNqSGNIdHhvU1dDTmoxb0I1WVJzVGdvWDhTbkRvOUhCWm1kUmR4LlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwiYWNjZXB0IjpbImRpZGNvbW0vdjIiLCJkaWRjb21tL2FpcDI7ZW52PXJmYzU4NyJdfX0


In [43]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
alice_did_for_mediator = await create_peer_did(1,1)
print("Alice's DID:", alice_did_for_mediator)

{'crv': 'Ed25519', 'x': 'bnSOSkiFNN3l4QMsVvMky2-vwihSgstnMtSJuQURtK4', 'd': 'PGhLWuwymxaBNAp4k_A_VVGpUNpBCcmg-p8eKBGyQWA', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSfenAn9X4NvoCnTCa8T9rMb3QfNmnwj7iJr71oTZnhGwU.Vz6MkmtRyVTJZsNqCffDHxuFcDvn3bk3PUaTh3jrFGE8oKFXT#6MkmtRyVTJZsNqCffDHxuFcDvn3bk3PUaTh3jrFGE8oKFXT'}
{'crv': 'X25519', 'x': 'OxMqa2e0b_bJYt-fKq8xhGjjbVf0oxbR6gxpgPaMRQM', 'd': '4Bi4dNuygBV8LGQiue0UJMB3K7mFGB0zM60MTMJOeV8', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSfenAn9X4NvoCnTCa8T9rMb3QfNmnwj7iJr71oTZnhGwU.Vz6MkmtRyVTJZsNqCffDHxuFcDvn3bk3PUaTh3jrFGE8oKFXT#6LSfenAn9X4NvoCnTCa8T9rMb3QfNmnwj7iJr71oTZnhGwU'}
Alice's DID: did:peer:2.Ez6LSfenAn9X4NvoCnTCa8T9rMb3QfNmnwj7iJr71oTZnhGwU.Vz6MkmtRyVTJZsNqCffDHxuFcDvn3bk3PUaTh3jrFGE8oKFXT


### Alice request mediate

In [44]:
received_msg_decoded

{'type': 'https://didcomm.org/out-of-band/2.0/invitation',
 'id': 'a2fc2863-f8c6-47f9-a3d3-8871ce704bab',
 'from': 'did:peer:2.Ez6LSo7greAQ4GZRuparQC1cysxubcCFjVVZdsvDSocEkL971.Vz6MktiFh7h3jHcHtxoSWCNj1oB5YRsTgoX8SnDo9HBZmdRdx.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0',
 'body': {'goal_code': 'request-mediate',
  'goal': 'RequestMediate',
  'accept': ['didcomm/v2', 'didcomm/aip2;env=rfc587']}}

In [45]:
alice_mediate_grant = Message(
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/coordinate-mediation/2.0/mediate-request",
    body = {}
)

In [46]:
alice_mediate_grant_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_mediate_grant,
    frm = alice_did_for_mediator,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending to Mediator

In [47]:
mediator_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_endpoint)

https://mediator.rootsid.cloud


In [48]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_mediate_grant_packed.packed_msg)


In [49]:
print(resp.json())

{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTNKMGIxcEJRekYzVUVKUWEyaDBlVzVDU0hsM05ITnlRMjlOTkZsTVVtdHBVR1ZLZWs1R1RHbG1RMFZrTGxaNk5rMXJkalJ0WW5OVlExQnpWamxxU0ZwRE9YSmFRMkZPYmtSamVHbG1SVWh1YVVOR1RsUktkMlpOUkhCdU1qRXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3eU1XeGFSMnhvWkVjNWVVeHVTblppTTFKNllWZFJkVmt5ZUhaa1YxRnBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGZ3dJelpNVTNKMGIxcEJRekYzVUVKUWEyaDBlVzVDU0hsM05ITnlRMjlOTkZsTVVtdHBVR1ZLZWs1R1RHbG1RMFZrIiwiYXB2IjoiZV9haU9wUmJhSG5UdkIwZWdrTkQ2OUxFRWw3NUdfYXQ0RFBtSW5CaXB3ayIsInNraWQiOiJkaWQ6cGVlcjoyLkV6NkxTcnRvWkFDMXdQQlBraHR5bkJIeXc0c3JDb000WUxSa2lQZUp6TkZMaWZDRWQuVno2TWt2NG1ic1VDUHNWOWpIWkM5clpDYU5uRGN4aWZFSG5pQ0ZOVEp3Zk1EcG4yMS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndjem92TDIxbFpHbGhkRzl5TG5KdmIzUnphV1F1WTJ4dmRXUWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTcn

### Get mediator grant response

In [50]:
mediator_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg.message.type)
print("Routing Key: " + mediator_unpack_msg.message.body["routing_did"])
print("Mediator new DID: " + mediator_unpack_msg.message.from_prior.sub)
# mediator rotated did
mediator_routing_key = mediator_unpack_msg.message.body["routing_did"]
mediator_did = mediator_unpack_msg.message.from_prior.sub

https://didcomm.org/coordinate-mediation/2.0/mediate-grant
Routing Key: did:peer:2.Ez6LSr138e2ciftyWTd9FZAxnK1BQjpqF4zMfQU6eiYaok3W2.Vz6Mkq1qL56jVPvK4HFJFcSgx85MSfewhrhiiUjo69SWQEfjR.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0
Mediator new DID: did:peer:2.Ez6LSrtoZAC1wPBPkhtynBHyw4srCoM4YLRkiPeJzNFLifCEd.Vz6Mkv4mbsUCPsV9jHZC9rZCaNnDcxifEHniCFNTJwfMDpn21.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0


### Keylist update

Alice create a new DID to share with  other peers using mediator routing keys 

In [51]:
#alice_did_new = await create_peer_did(1, 1, service_endpoint=mediator_endpoint, service_routing_keys=[mediator_routing_key])
alice_did_for_bob = await create_peer_did(1, 1, service_endpoint=[{"uri": mediator_routing_key}])

print("Alice's DID for Bob:", alice_did_for_bob)

{'crv': 'Ed25519', 'x': 'UWbvVNe2OB5bk1KNiG8uqKrJaKM_AWmk_vfBqPRACkk', 'd': 'JpOHbJBenfnB9_2EHsz-j5GO45yVd8UlK7JudxKJPVc', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSbrMX7yaWXf9hsqTb5xGH6qGdXLrESfHTXf3eq2NVMvjp.Vz6Mkjw273voGYF2gqHjWmvjsay7KQZ49LXnruyp3PNRbNdVz.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2TFNyMTM4ZTJjaWZ0eVdUZDlGWkF4bksxQlFqcHFGNHpNZlFVNmVpWWFvazNXMi5WejZNa3ExcUw1NmpWUHZLNEhGSkZjU2d4ODVNU2Zld2hyaGlpVWpvNjlTV1FFZmpSLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCJ9XSwiYSI6WyJkaWRjb21tL3YyIl19#6Mkjw273voGYF2gqHjWmvjsay7KQZ49LXnruyp3PNRbNdVz'}
{'crv': 'X25519', 'x': 'ApuERPbMOWEKbNClOrDuO_Uxl7eo-3oTfTlvkfnyoGc', 'd': 'CHCdY2ALS6poaWy2T_Q11YlWr2s7_5r9Z8P-bJiN_mY', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSbrMX7yaWXf9hsqTb5xGH6qGdXLrESfHTXf3eq2NVMvjp.Vz6Mkjw273voGYF2gqHjWmvjsay7KQZ49LXnruyp3PNRbNdVz.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIu

Alice create and send the keylist update message

In [56]:
alice_keylist_update = Message(
    id = "unique-id-293e9a922efff",
    type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
    body = {
        "updates":[
        {
            "recipient_did": alice_did_for_bob,
            "action": "add"
        }
    ]
    }
)

In [57]:
alice_keylist_update_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_keylist_update,
    frm = alice_did_for_mediator,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [58]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_keylist_update_packed_msg.packed_msg)

In [59]:
mediator_unpack_msg2 = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg2.message.type)
print(mediator_unpack_msg2.message.body)


https://didcomm.org/coordinate-mediation/2.0/keylist-update-response
{'updated': [{'recipient_did': 'did:peer:2.Ez6LSbrMX7yaWXf9hsqTb5xGH6qGdXLrESfHTXf3eq2NVMvjp.Vz6Mkjw273voGYF2gqHjWmvjsay7KQZ49LXnruyp3PNRbNdVz.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2TFNyMTM4ZTJjaWZ0eVdUZDlGWkF4bksxQlFqcHFGNHpNZlFVNmVpWWFvazNXMi5WejZNa3ExcUw1NmpWUHZLNEhGSkZjU2d4ODVNU2Zld2hyaGlpVWpvNjlTV1FFZmpSLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd2N6b3ZMMjFsWkdsaGRHOXlMbkp2YjNSemFXUXVZMnh2ZFdRaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCJ9XSwiYSI6WyJkaWRjb21tL3YyIl19', 'action': 'add', 'result': 'no_change'}]}


### Alice can now send her DID to Bob.

### Alice check status of queued messages from the Mediator

In [60]:
alice_status_check = Message(
    id = "unique-id-293e9a922efffxxx",
    type = "https://didcomm.org/messagepickup/3.0/status-request",
    body = {}
)

In [61]:
alice_status_check_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_status_check,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp3 = requests.post(mediator_endpoint, headers=headers, data = alice_status_check_packed_msg.packed_msg)

In [62]:
mediator_unpack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp3.json()
)
print(mediator_unpack_status.message.type)
print("Messages in Mediator queue: " + str(mediator_unpack_status.message.body["message_count"]))

https://didcomm.org/messagepickup/3.0/status
Messages in Mediator queue: 1


### If there are messages, Alice can pickup from the mediator (delivery request)

In [63]:
alice_delivery_request = Message(
    id = "unique-id-293e9a922efffxxxff",
    type = "https://didcomm.org/messagepickup/3.0/delivery-request",
    body = {"limit": 1}
)

In [64]:
alice_delivery_request_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_delivery_request,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp4 = requests.post(mediator_endpoint, headers=headers, data = alice_delivery_request_packed_msg.packed_msg)

In [65]:
mediator_delivery = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp4.json()
)
print(mediator_delivery.message.type)
print(mediator_delivery.message.body)

https://didcomm.org/messagepickup/3.0/delivery
{'recipient_key': None}


In [66]:
bob_packed_msg = mediator_delivery.message.attachments[0].data.json
msg_id = mediator_delivery.message.attachments[0].id

In [67]:
print(bob_packed_msg)

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTNSWmVFRnVRbHBqWW5JMGQzQlNXbkpEWmxVeGVHcFdkRlJZWjBkNWExZzNjWFEwWjFGd1VXTk9kbFo0TGxaNk5rMXJhR0YwUWtOeVIyRTJOa0ZtU0RRM05VcEhkSEJPUmtWYWRGZFVORzVSZG5OUmJqZHpWbkF6VkRoWVozUXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3elpETmtlVFZzWlVkR2RHTkhlR3hNYlU1MllsTTVhV0l5U1dsTVEwcG9TV3B3WWtsdFVuQmFSMDUyWWxjd2RtUnFTV2xZV0RBak5reFRkRmw0UVc1Q1dtTmljalIzY0ZKYWNrTm1WVEY0YWxaMFZGaG5SM2xyV0RkeGREUm5VWEJSWTA1MlZuZyIsImFwdiI6IlVKbHhRZGN5WmFhRUN4VDZfS3Q5TmNkU1ctRnQ5a2VFd2lfMk0wRmlEQzQiLCJza2lkIjoiZGlkOnBlZXI6Mi5FejZMU3RZeEFuQlpjYnI0d3BSWnJDZlUxeGpWdFRYZ0d5a1g3cXQ0Z1FwUWNOdlZ4LlZ6Nk1raGF0QkNyR2E2NkFmSDQ3NUpHdHBORkVadFdUNG5RdnNRbjdzVnAzVDhYZ3QuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwzZDNkeTVsZUdGdGNHeGxMbU52YlM5aWIySWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTdFl4QW5CWmNic

In [68]:
bob_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= bob_packed_msg
)

In [69]:
print("Message ID:", msg_id)
print(bob_msg.message)

Message ID: 632fc3039a393e8978b2ef20
Message(id='fc859a88-e0f0-4b45-a478-c7e1cae02ab9', type='https://didcomm.org/discover-features/2.0/queries', body={'queries': [{'feature-type': 'protocol', 'match': 'https://didcomm.org/basicmessage/2.*'}]}, frm=None, to=None, created_time=None, expires_time=None, from_prior=None, please_ack=None, ack=None, thid=None, pthid=None, attachments=None, custom_headers=[{'return_route': 'all'}])


In [70]:
alice_ack = Message(
    id = "unique-id-293e9a922efffxxxffsss",
    type = "https://didcomm.org/messagepickup/3.0/messages-received",
    body = {"message_id_list": [msg_id]}
)

In [71]:
alice_ack_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_ack,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp5 = requests.post(mediator_endpoint, headers=headers, data = alice_ack_packed_msg.packed_msg)

In [72]:
mediator_ack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp5.json()
)
print(mediator_ack_status.message.type)
print(mediator_ack_status.message.body)

https://didcomm.org/messagepickup/3.0/status
{'message_count': 0, 'live_delivery': False}
